# yolov5-face
* YOLOv5是一种单阶段目标检测算法，在这个样例中，我们选取了YOLOv5s，它是YOLOv5系列中较为轻量的网络，适合在边缘设备部署，进行实时目标检测。

# 前期准备
* 基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可以参考下面的步骤。
* 首先我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，下载我们已经准备好了的ONNX模型，ONNX是开源的离线推理模型框架。

* 为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用，以下为转换指令：  
    ```shell
    atc --model=yolov5s.onnx --framework=5 --output=yolo --input_format=NCHW --input_shape="input_image:1,3,640,640" --log=error --soc_version=Ascend310B1
    ```
    * 其中转换参数的含义为：  
        * --model：输入模型路径
        * --framework：原始网络模型框架类型，5表示ONNX
        * --output：输出模型路径
        * --input_format：输入Tensor的内存排列方式
        * --input_shape：指定模型输入数据的shape
        * --log：日志级别
        * --soc_version：昇腾AI处理器型号
        * --input_fp16_nodes：指定输入数据类型为FP16的输入节点名称
        * --output_type：指定网络输出数据类型或指定某个输出节点的输出类型

# 模型推理实现

In [1]:
# 导入代码依赖
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import torch
from skvideo.io import vreader, FFmpegWriter
import IPython.display
from ais_bench.infer.interface import InferSession

from det_utils import letterbox, scale_coords, nms, scale_coords_landmarks

/usr/local/miniconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [5]:
def preprocess_image(image, cfg, bgr2rgb=True):
    """图片预处理"""
    img, scale_ratio, pad_size = letterbox(image, new_shape=cfg['input_shape'])
    if bgr2rgb:
        img = img[:, :, ::-1]
    img = img.transpose(2, 0, 1)  # HWC2CHW
    img = np.ascontiguousarray(img, dtype=np.float32)
    img /= 255.0  # 关键修改：归一化 0-255 -> 0.0-1.0
    return img, scale_ratio, pad_size


def draw_bbox(bbox, img0, color, wt, names):
    """在图片上画预测框"""
    det_result_str = ''
    for idx in range(bbox.shape[0]):
        # 关键修改：yolov5-face NMS后，index 5 是类别
        class_id = int(bbox[idx][5]) 
        
        # 关键修改：修复之前的语法错误 float(bool) -> float(val)
        if float(bbox[idx][4]) < 0.05:
            continue
            
        img0 = cv2.rectangle(img0, (int(bbox[idx][0]), int(bbox[idx][1])), (int(bbox[idx][2]), int(bbox[idx][3])),
                             color, wt)
        
        label_name = names[int(class_id)] if names and len(names) > class_id else 'face'
        img0 = cv2.putText(img0, str(idx) + ' ' + label_name, (int(bbox[idx][0]), int(bbox[idx][1] + 16)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        img0 = cv2.putText(img0, '{:.4f}'.format(bbox[idx][4]), (int(bbox[idx][0]), int(bbox[idx][1] + 32)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
        # 关键修改：关键点从 index 6 开始
        if bbox.shape[1] >= 16:
            for k in range(5):
                kpt_x = int(bbox[idx][6 + 2 * k])
                kpt_y = int(bbox[idx][6 + 2 * k + 1])
                cv2.circle(img0, (kpt_x, kpt_y), 3, (255, 0, 0), -1)

        det_result_str += '{} {} {} {} {} {}\n'.format(
           label_name, str(bbox[idx][4]), bbox[idx][0], bbox[idx][1], bbox[idx][2], bbox[idx][3])
    return img0


def get_labels_from_txt(path):
    # ...existing code...
    labels_dict = dict()
    with open(path) as f:
        for cat_id, label in enumerate(f.readlines()):
            labels_dict[cat_id] = label.strip()
    return labels_dict


def draw_prediction(pred, image, labels):
    # ...existing code...
    imgbox = widgets.Image(format='jpg', height=720, width=1280)
    img_dw = draw_bbox(pred, image, (0, 255, 0), 2, labels)
    imgbox.value = cv2.imencode('.jpg', img_dw)[1].tobytes()
    display(imgbox)


def infer_image(img_path, model, class_names, cfg):
    """图片推理"""
    image = cv2.imread(img_path)
    img, scale_ratio, pad_size = preprocess_image(image, cfg)
    
    # 关键修改：增加 Batch 维度 (3,640,640) -> (1,3,640,640)
    if len(img.shape) == 3:
        img = img[None] 

    output = model.infer([img])[0]
    output = torch.tensor(output)
    
    # NMS nm=10 保留关键点
    boxout = nms(output, conf_thres=cfg["conf_thres"], iou_thres=cfg["iou_thres"], nm=10)
    pred_all = boxout[0].numpy()
    
    # 坐标还原
    scale_coords(cfg['input_shape'], pred_all, image.shape, ratio_pad=(scale_ratio, pad_size))
    if pred_all.shape[1] >= 16:
        # 关键点还原，传入 index 6:16
        pred_all[:, 6:16] = scale_coords_landmarks(cfg['input_shape'], pred_all[:, 6:16], image.shape, ratio_pad=(scale_ratio, pad_size))
        
    draw_prediction(pred_all, image, class_names)


def infer_frame_with_vis(image, model, labels_dict, cfg, bgr2rgb=True):
    img, scale_ratio, pad_size = preprocess_image(image, cfg, bgr2rgb)
    
    # 关键修改：增加 Batch 维度
    if len(img.shape) == 3:
        img = img[None]

    output = model.infer([img])[0]
    output = torch.tensor(output)
    
    boxout = nms(output, conf_thres=cfg["conf_thres"], iou_thres=cfg["iou_thres"], nm=10)
    pred_all = boxout[0].numpy()
    
    scale_coords(cfg['input_shape'], pred_all, image.shape, ratio_pad=(scale_ratio, pad_size))
    if pred_all.shape[1] >= 16:
        pred_all[:, 6:16] = scale_coords_landmarks(cfg['input_shape'], pred_all[:, 6:16], image.shape, ratio_pad=(scale_ratio, pad_size))
        
    img_vis = draw_bbox(pred_all, image, (0, 255, 0), 2, labels_dict)
    return img_vis


def img2bytes(image):
    """将图片转换为字节码"""
    return bytes(cv2.imencode('.jpg', image)[1])


def infer_video(video_path, model, labels_dict, cfg):
    """视频推理"""
    image_widget = widgets.Image(format='jpeg', width=800, height=600)
    display(image_widget)

    # 读入视频
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, img_frame = cap.read()
        if not ret:
            break
        # 对视频帧进行推理
        image_pred = infer_frame_with_vis(img_frame, model, labels_dict, cfg, bgr2rgb=True)
        image_widget.value = img2bytes(image_pred)


def infer_camera(model, labels_dict, cfg):
    """外设摄像头实时推理"""
    def find_camera_index():
        max_index_to_check = 10  # Maximum index to check for camera

        for index in range(max_index_to_check):
            cap = cv2.VideoCapture(index)
            if cap.read()[0]:
                cap.release()
                return index

        # If no camera is found
        raise ValueError("No camera found.")

    # 获取摄像头
    camera_index = find_camera_index()
    cap = cv2.VideoCapture(camera_index)
    # 初始化可视化对象
    image_widget = widgets.Image(format='jpeg', width=1280, height=720)
    display(image_widget)
    while True:
        # 对摄像头每一帧进行推理和可视化
        _, img_frame = cap.read()
        image_pred = infer_frame_with_vis(img_frame, model, labels_dict, cfg)
        image_widget.value = img2bytes(image_pred)

# 样例运行

* 初始化相关参数

In [6]:
cfg = {
    'conf_thres': 0.4,  # 模型置信度阈值，阈值越低，得到的预测框越多
    'iou_thres': 0.5,  # IOU阈值，高于这个阈值的重叠预测框会被过滤掉
    'input_shape': [640, 640],  # 模型输入尺寸
}

model_path = 'yolo_face.om'
label_path = './coco_names.txt'
# 初始化推理模型
model = InferSession(0, model_path)
labels_dict = {0: 'face'}

* 选择推理模式。"infer_mode"有三个取值：image, camera, video，分别对应图片推理、摄像头实时推理和视频推理。默认使用视频推理模式。
* 我们选取的样例是一个赛车视频，执行下面的代码后可以看到模型会对视频的每一帧进行推理，并将预测结果展示在画面上。

In [8]:
infer_mode = 'camera'  ## 如果修改为'camera' 则使用摄像头进行推理

if infer_mode == 'image':
    img_path = 'ImageforInfer.jpg'
    infer_image(img_path, model, labels_dict, cfg)
elif infer_mode == 'camera':
    infer_camera(model, labels_dict, cfg)
elif infer_mode == 'video':
    video_path = 'racing.mp4'
    infer_video(video_path, model, labels_dict, cfg)

Image(value=b'', format='jpeg', height='720', width='1280')

KeyboardInterrupt: 

# 样例总结与扩展
以上就是这个样例的全部内容了，值得关注的是在模型推理后有一步非常重要的后处理，就是非极大值抑制，即NMS，由于模型的原始预测结果会有非常多无效或重叠的预测框，我们需要通过NMS来进行过滤。再者，模型预测框的表示往往是一个标准化的结果，比如0到1之间，我们需要通过坐标转换将结果与原始图片的宽高对应上。